In [55]:
import numpy as np
import pandas as pd
import h5py

In [61]:
exp_matrix=pd.read_csv('arabidopsis_expression_matrix.txt',delimiter='\t')
exp_matrix

,Sample,ERX1946687,ERX1946688,ERX1946689,ERX1946691,ERX1946693,ERX1946694,ERX1946679,ERX1946697,ERX1946698,...,SRX7913302,SRX7913308,SRX7913305,SRX7949143,SRX7949151,SRX7949154,GSM4422802,SRX145599,SRX145601,SRX145572
0,AT1G01010,9.549938,9.290362,9.718265,9.320113,7.727772,8.890327,9.202206,7.516327,7.106299,...,10.046072,8.153307,9.191076,1.348066,7.961390,7.379849,8.368486,23.087263,25.072099,25.620284
1,AT1G01020,17.366722,16.424985,14.706285,13.632893,17.120003,16.216256,21.308792,16.277702,17.832624,...,8.658232,13.048400,8.251458,19.198215,17.618640,18.382744,21.664295,18.614628,18.288629,18.300576
2,AT1G01030,2.370682,3.456960,3.466671,4.023898,2.987786,2.447023,3.336052,2.156807,2.377292,...,1.748905,2.095745,1.106731,14.369617,19.183660,8.563981,3.349984,4.683484,4.407466,3.920962
3,AT1G01040,28.222950,31.680245,30.870991,25.515105,23.358496,23.505800,26.544297,23.940144,23.049474,...,12.883631,14.345799,9.572236,8.205194,8.752240,11.793794,16.420449,23.384185,20.335179,23.180630
4,AT1G01050,69.038528,70.013577,74.540153,85.164081,76.150845,78.994937,68.504111,79.356451,82.143959,...,73.980990,72.022104,86.217547,69.526206,53.303801,57.050528,75.563856,71.424183,72.085186,68.909228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37331,ATMG09730,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
37332,ATMG09740,0.093549,0.108543,0.230965,0.291324,0.099224,0.294433,0.043048,0.054354,0.112172,...,0.000000,0.389842,0.035197,35.595999,33.370597,46.303811,0.000000,0.000000,0.796520,0.063929
37333,ATMG09950,0.000000,0.321113,0.146969,0.000000,0.338006,0.260572,0.390184,0.000000,0.000000,...,0.599381,1.426437,0.336607,45.386140,41.798265,53.913076,0.000000,1.952781,1.064701,1.322610
37334,ATMG09960,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,9.202387,18.383343,14.205293,0.000000,0.579943,0.000000,0.519587


In [62]:
gene_annotation=pd.read_csv('data/TAIR10_gene_annotation.txt',delimiter='\t')
gene_annotation

,Gene stable ID,Gene name,Chromosome/scaffold name,Gene start (bp),Gene end (bp),Strand
0,AT3G11415,AT3G11415,3,3588684,3589685,1
1,AT1G31258,AT1G31258,1,11171225,11171835,1
2,AT5G24735,AT5G24735,5,8468331,8469262,-1
3,AT2G45780,AT2G45780,2,18854555,18855184,-1
4,AT2G42425,AT2G42425,2,17661834,17662358,-1
...,...,...,...,...,...,...
32828,AT4G25880,PUM6,4,13154708,13159382,1
32829,AT1G30814,AT1G30814,1,10942648,10944727,-1
32830,AT2G43120,AT2G43120,2,17927181,17929138,1
32831,AT5G16970,AER,5,5575973,5578086,-1


In [63]:
exclude_geneid=[]

chromosome_list=[]
gene_pos_list=[]
for gid in exp_matrix['Sample']:
    line=gene_annotation[gene_annotation['Gene stable ID']==gid]
    if line.empty:
        exclude_geneid.append(gid)
    else:       
        chrom,geneid,gene_start,gene_end,strand=line['Chromosome/scaffold name'].item(),line['Gene stable ID'].item(),\
            line['Gene start (bp)'].item(),line['Gene end (bp)'].item(),line['Strand'].item()
        chromosome_list.append([chrom,geneid])
        gene_pos_list.append([gene_start,gene_end,strand])
    

In [65]:
filtered_exp_matrix = exp_matrix[~exp_matrix['Sample'].isin(exclude_geneid)]
filtered_exp_matrix

,Sample,ERX1946687,ERX1946688,ERX1946689,ERX1946691,ERX1946693,ERX1946694,ERX1946679,ERX1946697,ERX1946698,...,SRX7913302,SRX7913308,SRX7913305,SRX7949143,SRX7949151,SRX7949154,GSM4422802,SRX145599,SRX145601,SRX145572
0,AT1G01010,9.549938,9.290362,9.718265,9.320113,7.727772,8.890327,9.202206,7.516327,7.106299,...,10.046072,8.153307,9.191076,1.348066,7.961390,7.379849,8.368486,23.087263,25.072099,25.620284
1,AT1G01020,17.366722,16.424985,14.706285,13.632893,17.120003,16.216256,21.308792,16.277702,17.832624,...,8.658232,13.048400,8.251458,19.198215,17.618640,18.382744,21.664295,18.614628,18.288629,18.300576
2,AT1G01030,2.370682,3.456960,3.466671,4.023898,2.987786,2.447023,3.336052,2.156807,2.377292,...,1.748905,2.095745,1.106731,14.369617,19.183660,8.563981,3.349984,4.683484,4.407466,3.920962
3,AT1G01040,28.222950,31.680245,30.870991,25.515105,23.358496,23.505800,26.544297,23.940144,23.049474,...,12.883631,14.345799,9.572236,8.205194,8.752240,11.793794,16.420449,23.384185,20.335179,23.180630
4,AT1G01050,69.038528,70.013577,74.540153,85.164081,76.150845,78.994937,68.504111,79.356451,82.143959,...,73.980990,72.022104,86.217547,69.526206,53.303801,57.050528,75.563856,71.424183,72.085186,68.909228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37331,ATMG09730,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
37332,ATMG09740,0.093549,0.108543,0.230965,0.291324,0.099224,0.294433,0.043048,0.054354,0.112172,...,0.000000,0.389842,0.035197,35.595999,33.370597,46.303811,0.000000,0.000000,0.796520,0.063929
37333,ATMG09950,0.000000,0.321113,0.146969,0.000000,0.338006,0.260572,0.390184,0.000000,0.000000,...,0.599381,1.426437,0.336607,45.386140,41.798265,53.913076,0.000000,1.952781,1.064701,1.322610
37334,ATMG09960,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,9.202387,18.383343,14.205293,0.000000,0.579943,0.000000,0.519587


In [66]:
np_matrix = filtered_exp_matrix.iloc[:, 1:].to_numpy()

In [75]:
with h5py.File('data/arabidopsis_expression_data.h5', 'w') as h5file:
    h5file['chrom_geneid']=np.string_(chromosome_list)
    h5file['pos_start_end_strand']=np.array(gene_pos_list)
    h5file['FPKM_matrix']=np_matrix 
    